In [1]:
#init_notebook_mode(connected=True)
%matplotlib inline
import matplotlib, plotly, fileinput
import matplotlib.pyplot as plt
import pandas as pd 
from collections import Counter

from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
dates = []
for line in fileinput.input('dates.txt'):
    line = line.rstrip()
    
    #ignore lines starting with # or ending with ?
    if line[0] == '#' or line[-1] == '?':
        continue
    
    _, date = line.split('|') # split lines
    date = date.replace('', '') # remove spaces
    dates.append(pd.Timestamp(date))
    
dates = sorted(dates)

In [3]:
colors = ['#eebcbc', '#eedeab', '#c8ecb5', '#bcc2f2', '#e8b6ee', '#b25a86', '#546e85']

#all data together
d = Counter([t.weekday_name for t in dates])
labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
all_values = [d[l] for l in labels]

empty_trace = go.Pie(labels=['None'], values=[100],
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name='None',
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))

main_trace = go.Pie(labels=labels, values=all_values,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name='Total',
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))

print('All', all_values, labels)

All [10, 12, 13, 11, 12, 9, 91] ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [4]:
#per decade
weekday_per_decade = {}

for t in dates:
    decade = (t.year//10)*10
    #print(decade, t.weekday_name)
    if decade not in weekday_per_decade:
        weekday_per_decade[decade] = []
    weekday_per_decade[decade].append(t.weekday_name)

#traces
traces = []
for decade in weekday_per_decade:
    c = Counter(weekday_per_decade[decade])
    weekday_per_decade[decade] = c
    
    current_values = []
    current_labels = []
    current_colors = []
    for j, l in enumerate(labels):
        if c[l] > 0:
            current_labels.append(l)
            current_values.append(c[l])
            current_colors.append(colors[j])
    
    g = go.Pie(labels=current_labels, values=current_values,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name=str(decade),
               marker=dict(colors=current_colors, 
                           line=dict(color='#000000', width=2)))
    traces.append(g)
    print(decade, current_values, current_labels)

traces += [main_trace]#, empty_trace]
#traces

1830 [1, 3, 2] ['Monday', 'Wednesday', 'Friday']
1840 [1, 2, 2, 1] ['Monday', 'Friday', 'Saturday', 'Sunday']
1850 [1, 1, 2, 2] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
1860 [2, 2, 1] ['Tuesday', 'Thursday', 'Friday']
1870 [1, 2, 1, 2] ['Monday', 'Thursday', 'Saturday', 'Sunday']
1880 [1, 2] ['Saturday', 'Sunday']
1890 [1, 2, 2] ['Wednesday', 'Friday', 'Sunday']
1900 [1, 1, 2] ['Monday', 'Thursday', 'Sunday']
1910 [3, 1, 1] ['Monday', 'Friday', 'Sunday']
1920 [2, 1] ['Tuesday', 'Sunday']
1930 [1, 7] ['Friday', 'Sunday']
1970 [1, 3, 2, 3] ['Monday', 'Wednesday', 'Thursday', 'Sunday']
1980 [6, 2, 4, 3, 1, 10] ['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
1990 [2, 1, 21] ['Tuesday', 'Saturday', 'Sunday']
2000 [1, 3, 1, 20] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
2010 [17] ['Sunday']


In [13]:
#button list

keys = list(weekday_per_decade.keys())
keys += ['Total']#, 'None']

button_list = []
for i, decade in enumerate(keys):
    #mask
    bools = [False]*len(keys)
    bools[i] = True
    if i != len(keys)-1:
        label = str(decade)
        title = 'Elections or referendums celebrated during the decade: {}'.format(decade)
    else:
        label = 'Total'
        title = 'All elections or referendums celebrated in total since 1830'
    #button
    button = {
        'label': label,
        'method': 'update',
        'args': [{'visible': bools}, 
              {'title': title}]
    }
    #add
    button_list.append(button)
    


In [33]:
updatemenus = list([
    dict(active=len(traces)-1,
         buttons=button_list,
    )
])

layout = dict(title='All elections or referendums celebrated in total since 1830', 
              showlegend=True, updatemenus=updatemenus, width=600, height=620,
             annotations=[
        dict(
            x=0.5,
            y=0,
            xshift=0,
            yshift=0,
            visible= True,
            align='center',
            xref='paper',
            yref='paper',
            text='Checkout the GitHub <a href="https://carlosvega.github.io/Weekday_ESP_Elections/">repository</a> for more',
            showarrow=False,
            arrowhead=0
        )
    ])

fig = dict(data=traces, layout=layout)

iplot(fig, filename='styled_pie_chart')
plot(fig, filename='docs/styled_pie_chart', auto_open=False)

/usr/local/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `docs/styled_pie_chart` didn't end with .html. Adding .html to the end of your file.



'file:///Users/carlosvega/wespe/docs/styled_pie_chart.html'

In [ ]:
1830 [1, 3, 2] ['Monday', 'Wednesday', 'Friday']
1840 [1, 2, 2, 1] ['Monday', 'Friday', 'Saturday', 'Sunday']
1850 [1, 1, 2, 2] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
1860 [2, 2, 1] ['Tuesday', 'Thursday', 'Friday']
1870 [1, 2, 1, 2] ['Monday', 'Thursday', 'Saturday', 'Sunday']
1880 [1, 2] ['Saturday', 'Sunday']
1890 [1, 2, 2] ['Wednesday', 'Friday', 'Sunday']
1900 [1, 1, 2] ['Monday', 'Thursday', 'Sunday']
1910 [3, 1, 1] ['Monday', 'Friday', 'Sunday']
1920 [2, 1] ['Tuesday', 'Sunday']
1930 [1, 7] ['Friday', 'Sunday']
1970 [1, 3, 2, 3] ['Monday', 'Wednesday', 'Thursday', 'Sunday']
1980 [6, 2, 4, 3, 1, 10] ['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
1990 [2, 1, 21] ['Tuesday', 'Saturday', 'Sunday']
2000 [1, 3, 1, 20] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
2010 [17] ['Sunday']